#### 1. 什么是动态代理
* 动态代理像一个门面一样, 可以伪装成任何接口的实现, 而且可以把所有的方法调用路由到一个handler-invoke()方法  
* 上面的概念, 表明Java的动态代理只能对一个接口代理

#### 2. 如何使用动态代理
1. 创建一个handler, 实现`java.lang.reflect.InvocationHandler`接口
```java
class MyHandler implements InvocationHandler{
    @Override
    public Object invoke(Object proxy, Method method, Object[] args) throws Throwable {
        System.out.println("调用了"+method.getName()+" 在"+ LocalDate.now());
        return true;
    }
}
```
2. 使用静态方法`Proxy.newProxyInstance()`创建代理实例  
    1. 参数:   
        * ClassLoader loader   
        * Class<?>[] interfaces : 代理的接口  
        * InvocationHandler h  
    2. 创建代理对象, 用代理对象执行方法  
        ```java
        List arr = (List) Proxy.newProxyInstance(MyHandler.class.getClassLoader(),
                        new Class[]{List.class},
                        new MyHandler());

        arr.add(1);
        /**
        输出: 调用了add 在2019-10-04
        */
        ```

3. 几种创建代理对象的错误
    1. 对某个类进行代理
    ```java
    List arr = (List)Proxy.newProxyInstance(MyHandler.class.getClassLoader(),
                new Class[]{ArrayList.class},
                new MyHandler());
    /**
    报错: Exception in thread "main" java.lang.IllegalArgumentException: java.util.ArrayList is not an interface
    */
    ```
    2. 试图把代理对象转成某个实体类的对象
    ```java
    List arr = (ArrayList)Proxy.newProxyInstance(MyHandler.class.getClassLoader(),
                new Class[]{List.class},
                new MyHandler());
    /**
    报错: Exception in thread "main" java.lang.ClassCastException: com.sun.proxy.$Proxy0 cannot be cast to java.util.ArrayList
    */
    ```

#### 3. Example: Timing动态代理
1. 创建Handler
2. 创建代理对象

```java  
class TimingHandler implements InvocationHandler{
    private final Object execObj;  // 真正执行代理方法的对象
    private final Map<String,Method> methods = new HashMap<>();

    public TimingHandler(Object execObj) {
        this.execObj = execObj;
        // getDeclaredMethods: 返回类声明的所有方法, 包括private, public, protected
        for (Method method : execObj.getClass().getDeclaredMethods()) {
            methods.put(method.getName(),method);
        }
    }

    @Override
    public Object invoke(Object proxy, Method method, Object[] args) throws Throwable {
        long start = System.nanoTime();
        Object returnVal = method.invoke(execObj, args);
        long end = System.nanoTime();
        System.out.println("执行:"+method.getName()+", 花费"+(end-start)+"微秒");
        return returnVal;
    }
}

public class Test{
    public static void main(String[] args) {
        String execObj = new String("abcdefg");
        CharSequence o = (CharSequence)Proxy.newProxyInstance(Test.class.getClassLoader(),
                new Class[]{CharSequence.class},
                new TimingHandler(execObj));
        System.out.println("第三个字符: " + o.charAt(3));
    }
}

/**
执行:charAt, 花费15103微秒
第三个字符: d
*/
```